In [47]:
import os
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

In [48]:
# load model

alexnet = torch.hub.load('pytorch/vision:v0.10.0', 'alexnet', pretrained=True)
params = {name: parameter for name, parameter in alexnet.named_parameters()}

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0


In [67]:
class AlexNet(nn.Module):
    def __init__(self, num_classes=1000, hidden_info={}):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size = 3, stride = 2, dilation=1, ceil_mode=False),

            nn.Conv2d(64, 192, kernel_size=5, stride=1, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size = 3, stride = 2, dilation=1, ceil_mode=False),

            nn.Conv2d(192, 384, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),

            nn.Conv2d(384, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),

            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size = 3, stride = 2, dilation=1, ceil_mode=False),

            nn.Flatten(),
        )

        self.classifier = nn.Sequential(
            nn.Dropout(0.5, inplace=False),
            nn.Linear(9216, 4096),
            nn.ReLU(inplace=True),

            nn.Dropout(0.5, inplace=False),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),

            nn.Linear(4096, num_classes)
        )

        self.hidden_info = hidden_info
        
    def forward(self, x):
        x = x.float()
        for i, child in enumerate(list(self.features.children())):
            x = child(x)
            if i in self.hidden_info.keys(): self.hidden_info[i].append(x)
        out = self.classifier(x)
        return out

In [68]:
model = AlexNet(hidden_info={0: []})
model.load_state_dict(params)

<All keys matched successfully>

In [69]:
X = np.random.normal(0, 1, size=(1, 3, 224, 224))
X = torch.from_numpy(X).float()

out = model(X)